In [1]:
import sqlite3
import pandas as pd

In [2]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None

In [3]:
conn = sqlite3.connect('parch-and-posey.db')

In [4]:
cursor = conn.cursor()
cursor.execute('''
select * from sqlite_master where type = "table";
''')
columns = [col[0] for col in cursor.description]
data = cursor.fetchall()
cursor.close()

In [5]:
pd.DataFrame(data, columns=columns)

,type,name,tbl_name,rootpage,sql
0,table,web_events,web_events,2,"CREATE TABLE web_events (\tid integer,\taccoun..."
1,table,sales_reps,sales_reps,7,"CREATE TABLE sales_reps (\tid integer,\tname b..."
2,table,region,region,222,"CREATE TABLE region (\tid integer,\tname bpchar)"
3,table,orders,orders,223,"CREATE TABLE orders (\tid integer,\taccount_id..."
4,table,accounts,accounts,583,"CREATE TABLE accounts (\tid integer,\tname bpc..."


In [6]:
pd.read_sql_query(sql='''
SELECT *
FROM orders;
''', con=conn)

,id,account_id,occurred_at,standard_qty,gloss_qty,poster_qty,total,standard_amt_usd,gloss_amt_usd,poster_amt_usd,total_amt_usd
0,1,1001,2015-10-06 17:31:14,123,22,24,169,613.77,164.78,194.88,973.43
1,2,1001,2015-11-05 03:34:33,190,41,57,288,948.10,307.09,462.84,1718.03
2,3,1001,2015-12-04 04:21:55,85,47,0,132,424.15,352.03,0.00,776.18
3,4,1001,2016-01-02 01:18:24,144,32,0,176,718.56,239.68,0.00,958.24
4,5,1001,2016-02-01 19:27:27,108,29,28,165,538.92,217.21,227.36,983.49
...,...,...,...,...,...,...,...,...,...,...,...
6907,6908,4501,2016-06-29 04:03:39,11,199,59,269,54.89,1490.51,479.08,2024.48
6908,6909,4501,2016-07-29 19:58:32,5,91,96,192,24.95,681.59,779.52,1486.06
6909,6910,4501,2016-08-27 00:58:11,16,94,82,192,79.84,704.06,665.84,1449.74
6910,6911,4501,2016-11-22 06:52:22,63,67,81,211,314.37,501.83,657.72,1473.92


# QUIZ QUESTION

Often there is confusion about the difference between WHERE and HAVING. Select all the statements that are true regarding HAVING and WHERE statements.

WHERE subsets the returned data based on a logical condition.

WHERE appears after the FROM, JOIN, and ON clauses, but before GROUP BY.

HAVING appears after the GROUP BY clause, but before the ORDER BY clause.

HAVING is like WHERE, but it works on logical statements involving aggregations.

Questions: HAVING

Task 1:

How many of the sales reps have more than 5 accounts that they manage?

In [11]:
pd.read_sql_query(sql='''
SELECT s.name, COUNT (*) count
FROM sales_reps s
JOIN accounts a
ON a.sales_rep_id = s.id
GROUP BY s.name
HAVING count > 5
ORDER BY count DESC;
''', con=conn)

,name,count
0,Georgianna Chisholm,15
1,Vernita Plump,11
2,Micha Woodford,11
3,Maryanna Fiorentino,11
4,Maren Musto,11
5,Earlie Schleusner,11
6,Dorotha Seawell,11
7,Calvin Ollison,11
8,Saran Ram,10
9,Moon Torian,10


In [86]:
pd.read_sql_query(sql='''
SELECT COUNT(*) num_reps_above5
FROM(SELECT s.id, s.name, COUNT(*) num_accounts
     FROM accounts a
     JOIN sales_reps s
     ON s.id = a.sales_rep_id
     GROUP BY s.id, s.name
     HAVING COUNT(*) > 5
     ORDER BY num_accounts) AS Table1;
''', con=conn)

,num_reps_above5
0,34


Solution:

SELECT s.id, s.name, COUNT(*) num_accounts
FROM accounts a
JOIN sales_reps s
ON s.id = a.sales_rep_id
GROUP BY s.id, s.name
HAVING COUNT(*) > 5
ORDER BY num_accounts;

SELECT COUNT(*) num_reps_above5
FROM(SELECT s.id, s.name, COUNT(*) num_accounts
     FROM accounts a
     JOIN sales_reps s
     ON s.id = a.sales_rep_id
     GROUP BY s.id, s.name
     HAVING COUNT(*) > 5
     ORDER BY num_accounts) AS Table1;

Task 2:

How many accounts have more than 20 orders?

In [87]:
pd.read_sql_query(sql='''
SELECT a.name, COUNT (*) count
FROM orders o
JOIN accounts a
ON o.account_id = a.id
GROUP BY a.name
HAVING count > 20
ORDER BY count DESC;
''', con=conn)

,name,count
0,Leucadia National,71
1,Sysco,68
2,Supervalu,68
3,Arrow Electronics,67
4,Mosaic,66
...,...,...
115,Raytheon,22
116,Jabil Circuit,22
117,Thrivent Financial for Lutherans,21
118,Performance Food Group,21


In [90]:
pd.read_sql_query(sql='''
SELECT COUNT(*) num_accounts_above20
FROM(SELECT a.id, a.name, COUNT(*) num_orders
FROM accounts a
JOIN orders o
ON a.id = o.account_id
GROUP BY a.id, a.name
HAVING COUNT(*) > 20
ORDER BY num_orders) AS Table1;
''', con=conn)

,num_accounts_above20
0,120


Solution:

Task 3:

Which account has the most orders?

In [13]:
pd.read_sql_query(sql='''
SELECT a.name, COUNT (*) count
FROM orders o
JOIN accounts a
ON o.account_id = a.id
GROUP BY a.name
ORDER BY count DESC
LIMIT 1;
''', con=conn)

,name,count
0,Leucadia National,71


Solution:

SELECT a.id, a.name, COUNT(*) num_orders
FROM accounts a
JOIN orders o
ON a.id = o.account_id
GROUP BY a.id, a.name
ORDER BY num_orders DESC
LIMIT 1;

Task 4:

Which accounts spent more than 30,000 usd total across all orders?

In [92]:
# BEWARE OF MISTAKE!!! - total columns in HAVING is in fact o.total!!!

pd.read_sql_query(sql='''
SELECT a.name, SUM(total_amt_usd) total
FROM orders o
JOIN accounts a
ON o.account_id = a.id
GROUP BY 1
HAVING total > 3000
ORDER BY 2 DESC;
''', con=conn)

,name,total
0,Paccar,161294.62


In [91]:
pd.read_sql_query(sql='''
SELECT a.name, SUM(total_amt_usd) total
FROM orders o
JOIN accounts a
ON o.account_id = a.id
GROUP BY 1
HAVING o.total > 3000
ORDER BY 2 DESC;
''', con=conn)

,name,total
0,Paccar,161294.62


In [30]:
# THIS IS CORRECT!

pd.read_sql_query(sql='''
SELECT a.name, SUM(total_amt_usd) total_USD
FROM orders o
JOIN accounts a
ON o.account_id = a.id
GROUP BY 1
HAVING total_USD > 30000
ORDER BY 2 DESC;
''', con=conn)

,name,total_USD
0,EOG Resources,382873.30
1,Mosaic,345618.59
2,IBM,326819.48
3,General Dynamics,300694.79
4,Republic Services,293861.14
...,...,...
199,Oracle,31231.56
200,Costco,30741.01
201,Group 1 Automotive,30708.92
202,PepsiCo,30095.72


In [96]:
pd.read_sql_query(sql='''
SELECT COUNT(*) num_accounts_above30t
FROM(SELECT a.id, a.name, SUM(o.total_amt_usd) total_spent
FROM accounts a
JOIN orders o
ON a.id = o.account_id
GROUP BY a.id, a.name
HAVING SUM(o.total_amt_usd) > 30000
ORDER BY total_spent) AS Table1;
''', con=conn)

,num_accounts_above30t
0,204


Solution:

SELECT a.id, a.name, SUM(o.total_amt_usd) total_spent
FROM accounts a
JOIN orders o
ON a.id = o.account_id
GROUP BY a.id, a.name
HAVING SUM(o.total_amt_usd) > 30000
ORDER BY total_spent;

Task 5:

Which accounts spent less than 1,000 usd total across all orders?

In [94]:
pd.read_sql_query(sql='''
SELECT a.name, SUM(total_amt_usd) total_USD
FROM orders o
JOIN accounts a
ON o.account_id = a.id
GROUP BY 1
HAVING total_USD < 1000
ORDER BY 2 DESC;
''', con=conn)

,name,total_USD
0,Level 3 Communications,881.73
1,Delta Air Lines,859.64
2,Nike,390.25


In [97]:
pd.read_sql_query(sql='''
SELECT COUNT(*) num_accounts_under1t
FROM(SELECT a.id, a.name, SUM(o.total_amt_usd) total_spent
FROM accounts a
JOIN orders o
ON a.id = o.account_id
GROUP BY a.id, a.name
HAVING SUM(o.total_amt_usd) < 1000
ORDER BY total_spent) AS Table1;
''', con=conn)

,num_accounts_under1t
0,3


Solution:

SELECT a.id, a.name, SUM(o.total_amt_usd) total_spent
FROM accounts a
JOIN orders o
ON a.id = o.account_id
GROUP BY a.id, a.name
HAVING SUM(o.total_amt_usd) < 1000
ORDER BY total_spent;

Task 6:

Which account has spent the most with us?

In [32]:
pd.read_sql_query(sql='''
SELECT a.name, SUM(total_amt_usd) total_USD
FROM orders o
JOIN accounts a
ON o.account_id = a.id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 1;
''', con=conn)

,name,total_USD
0,EOG Resources,382873.3


Solution:

SELECT a.id, a.name, SUM(o.total_amt_usd) total_spent
FROM accounts a
JOIN orders o
ON a.id = o.account_id
GROUP BY a.id, a.name
ORDER BY total_spent DESC
LIMIT 1;

Task 7:

Which account has spent the least with us?

In [33]:
pd.read_sql_query(sql='''
SELECT a.name, SUM(total_amt_usd) total_USD
FROM orders o
JOIN accounts a
ON o.account_id = a.id
GROUP BY 1
ORDER BY 2
LIMIT 1;
''', con=conn)

,name,total_USD
0,Nike,390.25


Solution:

SELECT a.id, a.name, SUM(o.total_amt_usd) total_spent
FROM accounts a
JOIN orders o
ON a.id = o.account_id
GROUP BY a.id, a.name
ORDER BY total_spent
LIMIT 1;

Task 8:

Which accounts used facebook as a channel to contact customers more than 6 times?

In [42]:
pd.read_sql_query(sql='''
SELECT a.name, COUNT(*) count
FROM web_events w
JOIN accounts a
ON w.account_id = a.id
WHERE w.channel = 'facebook'
GROUP BY 1
HAVING count > 6
ORDER BY 2 DESC;
''', con=conn)

,name,count
0,Gilead Sciences,16
1,TJX,15
2,AutoNation,14
3,Charter Communications,13
4,Philip Morris International,12
5,Marathon Petroleum,12
6,Disney,12
7,United Continental Holdings,11
8,Ecolab,11
9,EOG Resources,11


In [98]:
pd.read_sql_query(sql='''
SELECT COUNT(*) num_accounts_under1t
FROM(SELECT a.id, a.name, w.channel, COUNT(*) use_of_channel
FROM accounts a
JOIN web_events w
ON a.id = w.account_id
GROUP BY a.id, a.name, w.channel
HAVING COUNT(*) > 6 AND w.channel = 'facebook'
ORDER BY use_of_channel) AS Table1;
''', con=conn)

,num_accounts_under1t
0,46


Solution:

SELECT a.id, a.name, w.channel, COUNT(*) use_of_channel
FROM accounts a
JOIN web_events w
ON a.id = w.account_id
GROUP BY a.id, a.name, w.channel
HAVING COUNT(*) > 6 AND w.channel = 'facebook'
ORDER BY use_of_channel;

Task 9:

Which account used facebook most as a channel?

In [43]:
pd.read_sql_query(sql='''
SELECT a.name, COUNT(*) count
FROM web_events w
JOIN accounts a
ON w.account_id = a.id
WHERE w.channel = 'facebook'
GROUP BY 1
ORDER BY 2 DESC
LIMIT 1;
''', con=conn)

,name,count
0,Gilead Sciences,16


Solution:

SELECT a.id, a.name, w.channel, COUNT(*) use_of_channel
FROM accounts a
JOIN web_events w
ON a.id = w.account_id
WHERE w.channel = 'facebook'
GROUP BY a.id, a.name, w.channel
ORDER BY use_of_channel DESC
LIMIT 1;

Task 10:

Which channel was most frequently used by most accounts?

In [55]:
pd.read_sql_query(sql='''
SELECT a.name, w.channel, COUNT(*) count
FROM web_events w
JOIN accounts a
ON w.account_id = a.id
GROUP BY 1, 2
ORDER BY 3 DESC
LIMIT 50;
''', con=conn)

,name,channel,count
0,Leucadia National,direct,52
1,Colgate-Palmolive,direct,51
2,New York Life Insurance,direct,51
3,Philip Morris International,direct,49
4,ADP,direct,48
5,AutoNation,direct,48
6,BlackRock,direct,48
7,Charter Communications,direct,48
8,FirstEnergy,direct,48
9,Altria Group,direct,47


Solution:

SELECT a.id, a.name, w.channel, COUNT(*) use_of_channel
FROM accounts a
JOIN web_events w
ON a.id = w.account_id
GROUP BY a.id, a.name, w.channel
ORDER BY use_of_channel DESC
LIMIT 10;

In [85]:
pd.read_sql_query(sql='''
SELECT channel, COUNT(*) count
FROM(SELECT *
FROM(SELECT a.name, w.channel, COUNT(*) count
FROM web_events w
JOIN accounts a
ON w.account_id = a.id
GROUP BY 1, 2
ORDER BY 3 DESC) AS Table1
GROUP BY name) AS Table2
GROUP BY 1
ORDER BY 2 DESC;
''', con=conn)

,channel,count
0,direct,333
1,adwords,10
2,banner,5
3,organic,2
4,facebook,1


In [84]:
pd.read_sql_query(sql='''
SELECT *
FROM(SELECT a.name, w.channel, COUNT(*) count
FROM web_events w
JOIN accounts a
ON w.account_id = a.id
GROUP BY 1, 2
ORDER BY 3 DESC) AS Table1
GROUP BY name
HAVING channel = 'direct';
''', con=conn)

,name,channel,count
0,3M,direct,19
1,ADP,direct,48
2,AECOM,direct,8
3,AES,direct,2
4,AIG,direct,1
...,...,...,...
328,World Fuel Services,direct,4
329,Xcel Energy,direct,5
330,Xerox,direct,2
331,Yum Brands,direct,10


In [75]:
pd.read_sql_query(sql='''
SELECT *
FROM(SELECT a.name, w.channel, COUNT(*) count
FROM web_events w
JOIN accounts a
ON w.account_id = a.id
GROUP BY 1, 2
ORDER BY 3 DESC) AS Table1
GROUP BY name
HAVING channel != 'direct';
''', con=conn)

,name,channel,count
0,AmerisourceBergen,facebook,6
1,Ball,banner,1
2,Bed Bath & Beyond,organic,7
3,C.H. Robinson Worldwide,banner,1
4,CBS,adwords,1
5,Danaher,adwords,1
6,Delta Air Lines,adwords,2
7,Eli Lilly,adwords,3
8,Exxon Mobil,adwords,1
9,HCA Holdings,banner,2


In [104]:
# THE LEAST USED CHANNELS

pd.read_sql_query(sql='''
SELECT channel, COUNT(*) count
FROM(SELECT *
FROM(SELECT a.name, w.channel, COUNT(*) count
FROM web_events w
JOIN accounts a
ON w.account_id = a.id
GROUP BY 1, 2
--ORDER BY 3 DESC) AS Table1
ORDER BY 3) AS Table1
GROUP BY name) AS Table2
GROUP BY 1
ORDER BY 2 DESC;
''', con=conn)

,channel,count
0,banner,106
1,adwords,102
2,twitter,47
3,facebook,38
4,direct,31
5,organic,27
